In [42]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime, openpyxl
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords 
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

#df, code = generate_code_from_db(ctx, qgroup=4028)

In [71]:
ldf = pd.read_csv(f'{ctx.get_dest()}ldf.csv').loc[:,['col','icn']] 
df=sql_todf("""
SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
LEFT JOIN PickListTypes ON UsageField=COLUMN_NAME
WHERE TABLE_NAME = 'PQCInventory'  
""", connstr)   
 
df = normalizedf(df)
df = pd.merge(df,ldf, left_on='COLUMN_NAME', right_on='col')
#df.loc[df['PK_PickListType'] != '0']
#df


,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,PK_PickListType,col,icn
3,FIPS199SystemCategorization,INT,0,481,FIPS199SystemCategorization,FIPS 199 System Categorization
6,CryptoAlgorithmUsed,INT,0,475,CryptoAlgorithmUsed,Cryptographic Algorithm Used
8,LengthCryptoKeysModules,INT,0,476,LengthCryptoKeysModules,Length of associated cryptographic keys or mod...
9,SoftwarePackageType,INT,0,482,SoftwarePackageType,Software Package Type (COTS/GOTS/NDI/Other)
14,HostingInfo,INT,0,477,HostingInfo,Hosting Information
15,CloudServiceProviders,INT,0,478,CloudServiceProviders,Cloud Service Providers
16,GovernmentCloudProviders,INT,0,479,GovernmentCloudProviders,Government Cloud Providers
17,HybridCloudProviders,INT,0,480,HybridCloudProviders,Hybrid Cloud Providers


In [80]:

def get_datafield(df, out=False):
    txt=[]   
    for i,r in df.iterrows():
        plt=''
        PK_PickListType=r['PK_PickListType']
        COLUMN_NAME=r['COLUMN_NAME']
        icn=r['icn']
        DATA_TYPE=r['DATA_TYPE'] 
        if PK_PickListType != '0':
            plt=f' PK_PickListType="{PK_PickListType}" '
        s=f'<CB:DataField DBColumnName="{COLUMN_NAME}" ImportColumnName="{icn}" {plt} runat="server"/>'
        
        txt.append(f'{s}')
    txt='\n'.join(txt) 
    with open(f'{ctx.get_dest()}script.aspx', 'w', encoding='UTF-8') as fw:
        fw.write(txt)
    if out: print(txt)

def get_params(df,out=False):
    parms=[]   
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'] == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL" 
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME']])
    vals=', @'.join([c for c in df['COLUMN_NAME']]) 
    ins=f'\n\nINSERT INTO PQCInventory ({into})'
    ins=ins+f'\nVALUES (@{vals})'

    parms='\n'.join(parms)[1:]
  
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME']]) 
    sel=', '.join([f'{c}' for c in df['COLUMN_NAME']])
    
    with open(f'{ctx.get_dest()}script.sql', 'w', encoding='UTF-8') as fw:
        fw.write(parms)
 
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel }

def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    txt,ddl,dv,sel=[],[],[],[] 
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        for d in ldf: 
            col=d['col']
            s=t.replace("{col}",d['col']).replace("{ht}",d['icn'])
            sel.append(f',{col}')
            if d['PK_PickListType'] != '0':
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
            else:
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
            txt.append(f'{s}')     
    txt='\n'.join(txt) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:]
    with open(f'{ctx.get_dest()}script.aspx', 'w', encoding='UTF-8') as fw:
        fw.write(f'{txt} \n\n {ddl} \n\n {dv} \n\n {sel}')
    if out:  print(txt)
    return {'txt':txt, 'ddl':ddl,'dv':dv,'sel':sel}
 

def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df['DATA_TYPE']=df['DATA_TYPE'].str.upper() 
    df['PK_PickListType']=df['PK_PickListType'].fillna(0).astype(int)
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].fillna(0).astype(int)
    df[cols] = df[cols].astype(str)
    df=df.loc[:, ['COLUMN_NAME','DATA_TYPE','CHARACTER_MAXIMUM_LENGTH','PK_PickListType']]
    return df
 
#get_params(df, out=False)
#get_datafield(df, out=False) 
 

In [81]:
d=get_gridcols(df, out=False)
print ( d['sel'] )

AgencyName
,SystemName
,FISMASystemID
,FIPS199SystemCategorization
,HighValueAssetID
,CryptoModuleName
,CryptoAlgorithmUsed
,ServiceProvidedByCryptoModule
,LengthCryptoKeysModules
,SoftwarePackageType
,NameofVendorDev
,SoftwarePackageNames
,SoftwarePackageVersions
,OperatingSystem
,HostingInfo
,CloudServiceProviders
,GovernmentCloudProviders
,HybridCloudProviders
,EstFY2025BudgetReqsforFed
,EstFY2025BudgetReqsforCon
,EstFY2025BudgetReqsforSytemsInv
,EstTotalBudgetReqFY2025ReqforFed
,EstTotalBudgetReqFY2025ReqforCon
,EstTotalBudgetReqFY2025ReqforSytemsInv
,ProjectedSystemRefreshYear
,CrosscuttingCosts
,AdditionalNotes


In [74]:
d=get_gridcols(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\PQCInventory.ascx', 'r', encoding='UTF-8') as f:
    aspx=f.read()
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\PQCInventory.ascx', 'w', encoding='UTF-8') as f:
    aspx=re.sub('<!--cols-->(?s).*<!--~-->',f'<!--cols-->\n{d["txt"]}\n<!--~-->\n', aspx)
    f.write(aspx)